In [1]:
import pyspark as ps

In [2]:
sc = ps.SparkContext()

The data is from the US Department of Transport site
(http://www.transtats.bts.gov/databases.asp?Subject_ID=3&Subject_Desc=Passenger%20Travel&Mode_ID2=0)
This is flight performance data - looks similar to the data Jon is using. 

First step, create an RDD from the text file downloaded. Take a look at the data. 

In [3]:
airline = sc.textFile('492610425_T_ONTIME.csv')

In [4]:
airline.take(2)

[u'"YEAR","UNIQUE_CARRIER","ORIGIN_AIRPORT_ID","DEST_AIRPORT_ID","DEP_DELAY","ARR_DELAY","CANCELLED",',
 u'2015,"AA",12478,12892,-5.00,7.00,0.00,']

In [4]:
#Remove the quotes and the final , from each line. 
#Note the strip(',') basically replaces the comma at the end of each line. 

airline_no_quotes = airline.map(lambda line: line.replace('\'','').replace('\"','').strip(','))

In [6]:
airline_no_quotes.take(2)

[u'YEAR,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,ARR_DELAY,CANCELLED',
 u'2015,AA,12478,12892,-5.00,7.00,0.00']

### Timing
Some techniques to measure timing - can use %time magic function but better to import time and call 
time.time() at the start and end of the transaction. 

In [7]:
%time
airline_no_quotes.take(3)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


[u'YEAR,UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,ARR_DELAY,CANCELLED',
 u'2015,AA,12478,12892,-5.00,7.00,0.00',
 u'2015,AA,12478,12892,-10.00,-19.00,0.00']

In [8]:
import time
start_time = time.time()
airline_no_quotes.take(3)
end_time = time.time()
print "The time taken is {0}".format(end_time-start_time)

The time taken is 0.0356178283691


### Caching the RDD
You can cache the rdd using the rdd function cache()
The first time the Action command runs, the cache is populated so the next time you run the script the next steps will rely on the cached data. 

I presume you need to be careful which data you are caching....


In [5]:
airline_no_quotes.cache()

PythonRDD[2] at RDD at PythonRDD.scala:48

### Removing the header column
Now we want to extract the header column as a list for later and also to have a set of the data without the header column. 


In [6]:
header_line = airline_no_quotes.first()
header_list = header_line.split(',')
print header_list

[u'YEAR', u'UNIQUE_CARRIER', u'ORIGIN_AIRPORT_ID', u'DEST_AIRPORT_ID', u'DEP_DELAY', u'ARR_DELAY', u'CANCELLED']


In [7]:
airline_no_header = airline_no_quotes.filter(lambda line: line != header_line)

In [12]:
airline_no_header.first()

u'2015,AA,12478,12892,-5.00,7.00,0.00'


###Preparing the data in a better format

So what we want now is to convert our tabular like data into a list or set of rows so we can more easily process them. 
This will look something like below for each row: 
[u'UNIQUE_CARRIER',AA 
 u'FL_NUM',1
 u'ORIGIN_AIRPORT_ID',12478
 u'ORIGIN_CITY_NAME',New York, NY
 u'DEST_AIRPORT_ID',12892
 u'DEST_CITY_NAME',Los Angeles
 u'DEP_DELAY',-5.00
 u'ARR_DELAY',7.00
 u'CANCELLED', 0.00]

In [8]:
# A new transformation function that is a little bigger than a lambda that needs to do: 
# take each row and split the row into a list and then zip the list with the header list. 
# So each row is transposed into one column and the headings are attached, like a dictionary. 
def make_row(row):
    row_list = row.split(',')
    d = dict(zip(header_list,row_list))
    return d

In [15]:
#Test the function on the first line - it seems to do what we want
#make_row(airline_no_header.first())
print airline_no_header.first().split(',')
print header_list
dict(zip(header_list,airline_no_header.first().split(',')))

[u'2015', u'AA', u'12478', u'12892', u'-5.00', u'7.00', u'0.00']
[u'YEAR', u'UNIQUE_CARRIER', u'ORIGIN_AIRPORT_ID', u'DEST_AIRPORT_ID', u'DEP_DELAY', u'ARR_DELAY', u'CANCELLED']


{u'ARR_DELAY': u'7.00',
 u'CANCELLED': u'0.00',
 u'DEP_DELAY': u'-5.00',
 u'DEST_AIRPORT_ID': u'12892',
 u'ORIGIN_AIRPORT_ID': u'12478',
 u'UNIQUE_CARRIER': u'AA',
 u'YEAR': u'2015'}

In [9]:
# Now run the function over all rows and create a new rdd 
airline_rows = airline_no_header.map(make_row)

### dictionaries: 
The beauty of what we have just done is we have converted each row into a dictionary of key values. 
A dictionary is like a map. 
Accessing the fields is easy: 

airline_rows.first()['ARR_DELAY'] or  airline_rows.first().get('ARR_DELAY') returns the first row as a dictionary and then gets the value for that key. 
All rows in a dict are defined by the key fields. 

In [10]:
airline_rows.take(2)

[{u'ARR_DELAY': u'7.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-5.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'ORIGIN_AIRPORT_ID': u'12478',
  u'UNIQUE_CARRIER': u'AA',
  u'YEAR': u'2015'},
 {u'ARR_DELAY': u'-19.00',
  u'CANCELLED': u'0.00',
  u'DEP_DELAY': u'-10.00',
  u'DEST_AIRPORT_ID': u'12892',
  u'ORIGIN_AIRPORT_ID': u'12478',
  u'UNIQUE_CARRIER': u'AA',
  u'YEAR': u'2015'}]

### Aggregating the worst and best airlines
Which Airline has the worst arrival delays:
Create a key value pair of the destination airline and the arrival delay 
Do this by creating a new RDD that reads every row and extracts the from dict the field u'DEST_AIRPORT_ID as the key and the ARR_DELAY as the value

Note we are doing a few new things here: 
1. Adding some error handling when we convert to float: reads as "float(blah blah) if row[] is not null else 0"
2. We are using groupByKey().mapValues() where the groupby returns the groups and the mapValues has access to the data within each group. Look carefully at the mapValues() to grasp this concept.
3. numpy is used to perform the mean calculation


In [18]:
destination_keyvalue = airline_rows.map(lambda row: (row['DEST_AIRPORT_ID'],\
                                                     float(row['ARR_DELAY']) if row['ARR_DELAY'] else 0 ))

In [19]:
origin_keyvalue = airline_rows.map(lambda row: (row['ORIGIN_AIRPORT_ID'], \
                                                float(row['DEP_DELAY']) if row['DEP_DELAY'] else 0))

In [24]:
import numpy as np

mean_delays_dest = destination_keyvalue.groupByKey().mapValues(lambda delays: np.mean(delays.data))
mean_delays_origin = origin_keyvalue.groupByKey().mapValues(lambda delays: np.mean(delays.data))

In [25]:
mean_delays_dest.take(4)
mean_delays_origin.take(4)

[(u'11193', 8.3105662580268529),
 (u'13830', 4.7592496091714436),
 (u'15016', 7.7876410004904368),
 (u'14679', 6.6169898126848503)]

In [31]:
# now we want to find the best and worst airports so we use sortBy
# sortBy needs to be passed something to sort and also the ordering ASC or DESC
print "Dest airport: 5 best early arrivals \n"
print mean_delays_dest.sortBy(lambda means: means[1], ascending=True).take(5)

print "\n Dest airport: 5 best worst arrivals \n"
print mean_delays_dest.sortBy(lambda means: means[1], ascending=False).take(5)

print "\n Origin airport: 5 best early departures \n"
print mean_delays_origin.sortBy(lambda means: means[1], ascending=True).take(5)

print "\n Dest airport: 5 best worst departures \n"
print mean_delays_origin.sortBy(lambda means: means[1], ascending=False).take(5)

Dest airport: 5 best early arrivals 

[(u'10165', -24.555555555555557), (u'11092', -15.055555555555555), (u'13127', -9.7547169811320753), (u'15582', -6.716981132075472), (u'12343', -6.6037735849056602)]

 Dest airport: 5 best worst arrivals 

[(u'13459', 44.07692307692308), (u'13964', 27.941176470588236), (u'10268', 27.051724137931036), (u'11067', 26.8125), (u'11203', 26.733333333333334)]

 Origin airport: 5 best early departures 

[(u'11092', -8.8888888888888893), (u'11525', -8.8524590163934427), (u'10170', -8.5333333333333332), (u'10926', -8.3333333333333339), (u'15991', -7.8833333333333337)]

 Dest airport: 5 best worst departures 

[(u'11447', 31.26530612244898), (u'12335', 30.706896551724139), (u'13964', 28.588235294117649), (u'10930', 28.399999999999999), (u'12519', 28.373333333333335)]
